# Kaggle Agents + MLE-bench Lite (Colab)

Este notebook executa o `kaggle-agents` em competições do **MLE-bench Lite** e valida as submissões com `mlebench grade-sample`.

Objetivo:
- Rodar uma competição (ou o Lite) no MLE-bench
- Gerar `results.json`, `summary.json`, `results.csv`
- Comparar com o benchmark via `any_medal_percentage` (Low/Lite)

Pré-requisitos (Colab Secrets):
- `OPENAI_API_KEY`
- `KAGGLE_USERNAME`
- `KAGGLE_KEY`


## 1) Setup (clone + install)

Este notebook assume os paths:
- `/content/kaggle-agents`
- `/content/mle-bench`


In [ ]:
# GPU (opcional)
!nvidia-smi || echo "No GPU available (CPU mode)"

# Clone repos (requer rede)
!test -d /content/kaggle-agents || git clone https://github.com/gustavogomespl/kaggle-agents.git /content/kaggle-agents
!test -d /content/mle-bench || git clone https://github.com/openai/mle-bench.git /content/mle-bench

%cd /content/kaggle-agents
!ls -la | head

Tue Feb  3 19:52:21 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   33C    P8             11W /   72W |       0MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
# Instalar deps
!pip -q install uv
!uv pip install --system -e /content/kaggle-agents
!pip -q install -e /content/mle-bench

import sys
print('python:', sys.version)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.8/22.8 MB 123.4 MB/s eta 0:00:00
Using Python 3.12.12 environment at: /usr
Resolved 213 packages in 1.15s
Prepared 43 packages in 2.01s
Uninstalled 1 package in 1ms
Installed 43 packages in 68ms
 + anthropic==0.77.1
 + asyncer==0.0.8
 + backoff==2.2.1
 + bcrypt==5.0.0
 + build==1.4.0
 + catboost==1.2.8
 + category-encoders==2.9.0
 + chromadb==1.4.1
 + coloredlogs==15.0.1
 + colorlog==6.10.1
 + dataclasses-json==0.6.7
 + diskcache==5.6.3
 + dspy==3.1.2
 + dspy-ai==3.1.2
 + durationpy==0.10
 + fastuuid==0.14.0
 + feature-engine==1.9.3
 + filetype==1.2.0
 + gepa==0.0.24
 + humanfriendly==10.0
 + json-repair==0.56.0
 + kaggle-agents==0.1.0 (from file:///content/kaggle-agents)
 + kubernetes==35.0.0
 + langchain-anthropic==1.3.1
 + langchain-classic==1.0.1
 + langchain-community==0.4.1
 + langchain-google-genai==4.2.0
 + langchain-openai==1.1.7
 + langchain-text-splitters==1.1.0
 + litellm==1.81.7
 + marshmallow==3.26.2
 + mypy-extensions==1.1.0

## 2) Configuração (.env + Kaggle credentials)

Configure no Colab → **Secrets**:
- `OPENAI_API_KEY`
- `KAGGLE_USERNAME`
- `KAGGLE_KEY`

Opcional (para controle de custo/qualidade):
- `LLM_MODEL`, `PLANNER_MODEL`, `DEVELOPER_MODEL`, `EVALUATOR_MODEL`


In [ ]:
import os, json
from pathlib import Path

from google.colab import userdata

# ===== LLM config (ajuste aqui) =====
os.environ['LLM_PROVIDER'] = os.environ.get('LLM_PROVIDER', 'gemini')
os.environ['LLM_MODEL'] = os.environ.get('LLM_MODEL', 'gemini-3-flash-preview')
os.environ['LLM_TEMPERATURE'] = os.environ.get('LLM_TEMPERATURE', '0')
os.environ['LLM_MAX_TOKENS'] = os.environ.get('LLM_MAX_TOKENS', '16000')

# Per-role overrides (opcional)
os.environ['PLANNER_MODEL'] = os.environ.get('PLANNER_MODEL', os.environ['LLM_MODEL'])
os.environ['DEVELOPER_MODEL'] = os.environ.get('DEVELOPER_MODEL', os.environ['LLM_MODEL'])
os.environ['EVALUATOR_MODEL'] = os.environ.get('EVALUATOR_MODEL', os.environ['LLM_MODEL'])

# ===== Secrets =====
os.environ['OPENAI_API_KEY'] = userdata.get('OPENAI_API_KEY') or ''
os.environ['GOOGLE_API_KEY'] = userdata.get('GEMINI_API_KEY') or ''
kaggle_username = userdata.get('KAGGLE_USERNAME') or ''
kaggle_key = userdata.get('KAGGLE_KEY') or ''

if not os.environ['OPENAI_API_KEY']:
    raise ValueError('Missing OPENAI_API_KEY (configure nos Colab Secrets).')
if not kaggle_username or not kaggle_key:
    raise ValueError('Missing KAGGLE_USERNAME/KAGGLE_KEY (configure nos Colab Secrets).')

# Kaggle credentials para o CLI do mlebench
kaggle_path = Path.home() / '.kaggle' / 'kaggle.json'
kaggle_path.parent.mkdir(parents=True, exist_ok=True)
kaggle_path.write_text(json.dumps({'username': kaggle_username, 'key': kaggle_key}))
kaggle_path.chmod(0o600)

# .env (útil para processos subprocess / reexecuções)
env_path = Path('/content/kaggle-agents/.env')
env_path.write_text(
    "\n".join([
        f"LLM_PROVIDER={os.environ['LLM_PROVIDER']}",
        f"LLM_MODEL={os.environ['LLM_MODEL']}",
        f"LLM_TEMPERATURE={os.environ['LLM_TEMPERATURE']}",
        f"LLM_MAX_TOKENS={os.environ['LLM_MAX_TOKENS']}",
        f"PLANNER_MODEL={os.environ['PLANNER_MODEL']}",
        f"DEVELOPER_MODEL={os.environ['DEVELOPER_MODEL']}",
        f"EVALUATOR_MODEL={os.environ['EVALUATOR_MODEL']}",
        f"OPENAI_API_KEY={os.environ['OPENAI_API_KEY']}",
        f"KAGGLE_USERNAME={kaggle_username}",
        f"KAGGLE_KEY={kaggle_key}",
        f"GOOGLE_API_KEY={os.environ['GOOGLE_API_KEY']}",
        "KAGGLE_AUTO_SUBMIT=false",
        "LOG_LEVEL=INFO",
        "LOG_DIR=./logs",
    ]) + "\n"
)

print('✅ kaggle.json:', kaggle_path)
print('✅ .env:', env_path)
print('✅ LLM_MODEL:', os.environ['LLM_MODEL'])

✅ kaggle.json: /root/.kaggle/kaggle.json
✅ .env: /content/kaggle-agents/.env
✅ LLM_MODEL: gemini-3-flash-preview


In [ ]:
print("Configuring LangSmith Tracing...")

try:
    from google.colab import userdata
    langchain_api_key = userdata.get('LANGSMITH')
    langsmith_project = userdata.get('LANGSMITH_PROJECT')
    print("✓ Loaded secrets from Colab")
except:
    langchain_api_key = None

if not langchain_api_key:
    langchain_api_key = getpass("Enter LangSmith API Key (optional): ")

if langchain_api_key:
    os.environ["LANGCHAIN_TRACING_V2"] = "true"
    os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
    os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
    os.environ["LANGCHAIN_PROJECT"] = langsmith_project
    print("✓ LangSmith Tracing Enabled (Project: kaggle-agents-tcc)")

    # Update .env file if it exists
    if os.path.exists(".env"):
        with open(".env", "a") as f:
            f.write(f"\nLANGCHAIN_TRACING_V2=true\n")
            f.write(f"LANGCHAIN_ENDPOINT=https://api.smith.langchain.com\n")
            f.write(f"LANGCHAIN_API_KEY={langchain_api_key}\n")
            f.write(f"LANGCHAIN_PROJECT={langsmith_project}\n")
else:
    print("ℹ️ LangSmith Tracing Disabled")

Configuring LangSmith Tracing...
✓ Loaded secrets from Colab
✓ LangSmith Tracing Enabled (Project: kaggle-agents-tcc)


## 3) Preparar datasets (MLE-bench)

Dica:
- Primeiro rode 1 competição para validar o pipeline.
- Depois rode `--lite` se quiser o Lite completo (22 competições).


In [ ]:
COMPETITION = "histopathologic-cancer-detection"

In [ ]:
# Teste rápido (1 competição)
# !mlebench prepare -c histopathologic-cancer-detection

# Lite completo (22 competições) - pode demorar
# !mlebench prepare --lite

In [ ]:
!mlebench prepare -c {COMPETITION} --keep-raw

[2026-02-03 19:53:07,481] [data.py:213] Downloading the dataset for `histopathologic-cancer-detection` to `/root/.cache/mle-bench/data/histopathologic-cancer-detection`...
100% 6.31G/6.31G [00:42<00:00, 207MB/s]
100% 6.31G/6.31G [00:42<00:00, 160MB/s]
[2026-02-03 19:53:50,848] [data.py:67] Generating checksum for `/root/.cache/mle-bench/data/histopathologic-cancer-detection/histopathologic-cancer-detection.zip`...
1.65MB [00:15, 110kB/s]                
[2026-02-03 19:54:05,875] [data.py:80] Checksum for `/root/.cache/mle-bench/data/histopathologic-cancer-detection/histopathologic-cancer-detection.zip` matches the expected checksum.
[2026-02-03 19:54:05,875] [data.py:83] Extracting `/root/.cache/mle-bench/data/histopathologic-cancer-detection/histopathologic-cancer-detection.zip` to `/root/.cache/mle-bench/data/histopathologic-cancer-detection/raw`...
[2026-02-03 19:55:35,009] [data.py:85] Extracted `/root/.cache/mle-bench/data/histopathologic-cancer-detection/histopathologic-cancer-de

## 4) Rodar avaliação (gera `results.json`, `summary.json`, `results.csv`)

Dica para economizar tempo/custo:
- Comece com `--max-iterations 1` e `--timeout 1200` (20 min por componente)
- Depois aumente gradualmente.


In [ ]:
from pathlib import Path

RUN_ID = 'aerial_cactus_smoke'
OUT_DIR = Path(f'/content/mlebench_results/{RUN_ID}')
OUT_DIR.mkdir(parents=True, exist_ok=True)

# Comando com timeout de componente alinhado às correções
!KAGGLE_AGENTS_TARGET_SCORE=0.99 \
KAGGLE_AGENTS_MAX_COMPONENT_RETRIES=3 \
KAGGLE_AGENTS_FAST_MODE=0 \
TESTING_TIMEOUT=8000 \
TIMEOUT_MODEL_HEAVY=8000 \
KAGGLE_AGENTS_COMPONENT_TIMEOUT_S=8000 \
KAGGLE_AGENTS_MAX_COMPONENTS=6 \
KAGGLE_AGENTS_ENABLE_LIMITS=false \
KAGGLE_AGENTS_CV_FOLDS=7 \
KAGGLE_AGENTS_HPO_USE_PRUNER=true \
KAGGLE_AGENTS_SOTA_BASE_K=5 \
KAGGLE_AGENTS_SOTA_EXPANDED_K=10 \
KAGGLE_AGENTS_FORCE_DATA_TYPE=text-normalization \
python3 notebooks/mlebench_eval.py \
--competition histopathologic-cancer-detection \
--max-iterations 12 \
--timeout 8000 \
-o {OUT_DIR}

[mlebench_eval] Starting evaluation at 2026-02-03 20:05:16.815651
MLE-BENCH EVALUATION
Competitions: 1
Max iterations: 12
Timeout per component: 8000s

######################################################################
# [1/1] histopathologic-cancer-detection
######################################################################
  Problem type: binary_classification
  Metric: auc
  Calling solve_mlebench()...
Colab environment detected, using: /content/kaggle_competitions
[MLEBenchDataAdapter] Detecting cache path...
[MLEBenchDataAdapter]   Path.home() = /root
[MLEBenchDataAdapter]   Checking /root/.cache/mle-bench/data, exists = True
[MLEBenchDataAdapter] Using cache path: /root/.cache/mle-bench/data
╭──────────────────────────────────────────────────────────────────────────────╮
│                                                                              │
│ MLE-BENCH MODE                                                               │
│                                         

In [ ]:
# Caminho do seu submission.csv (gerado pelo agente)
SUBMISSION_PATH = "/content/kaggle_competitions/competitions/histopathologic-cancer-detection/submission.csv"
COMPETITION = "histopathologic-cancer-detection"

# Grading com mlebench
!mlebench grade-sample {SUBMISSION_PATH} {COMPETITION}

[2026-02-03 19:58:11,558] [grade.py:69] Invalid submission file: /content/kaggle_competitions/competitions/histopathologic-cancer-detection/submission.csv. Please check that the file exists and it is a CSV.
[2026-02-03 19:58:11,570] [cli.py:202] Competition report:
[2026-02-03 19:58:11,570] [cli.py:203] {
    "competition_id": "histopathologic-cancer-detection",
    "score": null,
    "gold_threshold": 0.9835,
    "silver_threshold": 0.9798,
    "bronze_threshold": 0.9738,
    "median_threshold": 0.9477,
    "any_medal": false,
    "gold_medal": false,
    "silver_medal": false,
    "bronze_medal": false,
    "above_median": false,
    "submission_exists": false,
    "valid_submission": false,
    "is_lower_better": false,
    "created_at": "2026-02-03T19:58:11.569830",
    "submission_path": "/content/kaggle_competitions/competitions/histopathologic-cancer-detection/submission.csv"
}
